In [33]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import networkx as nx 
import pickle as pkl

import h5py

import sys
sys.path.append('../src/')
%aimport utils

# Overview 

(This is on the back-burner; worried it's too cyclic... data leak)
We will use the LINCS lvl 5 data to create a drug-protein MOA graph. 

We will create a similarity matrix between drugs (rows) and genetic perturbations (shRNA, crispr, cDNA), where similarity is calculated by transcriptional profile distance metrics. 

This is a naive approach to inferring drug targets and is intended to be a stand-in for more refined methods later. 

In [35]:
cmp_data, gene_sel = utils.get_lvl5_cmp_data()
cmp_data.head()

C:\Users\natha\miniconda3\envs\gcnvae\lib\site-packages\IPython\core\interactiveshell.py:3347: DtypeWarning: Columns (0,3,4,28,29) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,ADA,CDH2,AKT3,MED6,ACOT8,ABI1,GNPDA1,ZBTB33,CDH3,TANK,...,sig_id,pert_type,cell_iname,det_wells,det_plates,distil_ids,build_name,project_code,cmap_name,is_ncs_exemplar
53972,1.393475,-0.938388,0.033532,0.776472,-0.587603,2.055929,2.599445,0.992974,1.755998,0.490936,...,ABY001_A375_XH:BRD-A61304759:10:24,trt_cp,A375,H13|H17|H21,ABY001_A375_XH_X1_B15,ABY001_A375_XH_X1_B15:H13|ABY001_A375_XH_X1_B1...,NaN,ABY,tanespimycin,0
35118,0.540987,-0.331108,-1.019858,-1.025045,-0.724501,-0.497458,2.144441,0.731610,1.299909,0.294763,...,ABY001_A375_XH:BRD-K19687926:10:24,trt_cp,A375,A16|A20|A24,ABY001_A375_XH_X1_B15,ABY001_A375_XH_X1_B15:A16|ABY001_A375_XH_X1_B1...,NaN,ABY,lapatinib,0
19148,1.040050,-1.871763,-0.345414,-2.843074,2.428420,0.618698,-2.903616,2.932698,1.715469,1.445842,...,ABY001_A375_XH:BRD-K21680192:10:24,trt_cp,A375,G13|G17|G21,ABY001_A375_XH_X1_B15,ABY001_A375_XH_X1_B15:G13|ABY001_A375_XH_X1_B1...,NaN,ABY,mitoxantrone,0
42730,0.610513,-1.868991,-0.478798,-1.505409,-0.710490,0.274694,9.111989,-0.225553,-0.656211,0.142181,...,ABY001_A375_XH:BRD-K66175015:10:24,trt_cp,A375,A13|A17|A21,ABY001_A375_XH_X1_B15,ABY001_A375_XH_X1_B15:A13|ABY001_A375_XH_X1_B1...,NaN,ABY,afatinib,0
47917,0.330001,-0.531912,0.675849,0.897185,0.235430,-0.995797,-0.277181,-0.121663,0.504133,0.854838,...,ABY001_A375_XH:BRD-K70401845:10:24,trt_cp,A375,A14|A18|A22,ABY001_A375_XH_X1_B15,ABY001_A375_XH_X1_B15:A14|ABY001_A375_XH_X1_B1...,NaN,ABY,erlotinib,0
